# 🏊‍♀️ MVP — Classificação de Medalhistas em Natação Olímpica (1912–2020)

**Autora:** Luiza Oliveira Lima  
**Objetivo:** Prever se um(a) atleta conquistará **medalha** (ouro, prata, bronze) em provas olímpicas de natação, a partir de atributos da prova/atleta.

**Resumo executivo:** Utilizamos o dataset público *Olympic Swimming History (1912–2020)*. O melhor desempenho foi obtido com **XGBoost**. Após *tuning* e ajuste do **threshold = 0.40**, priorizamos **recall** para medalhistas, mantendo acurácia competitiva. Variáveis mais relevantes: **Team (país)**, **Relay?**, **Stroke**, **Distance**, **Year**.

> **Reprodutibilidade:** links de dados via URL, seeds fixas, pipelines `sklearn`, RandomizedSearchCV (CV=5). Notebook executa do início ao fim.


## 1. Importações, Seeds & Config


In [ ]:
RANDOM_STATE = 42
THRESHOLD = 0.40
TOP_N = 10

import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
np.random.seed(RANDOM_STATE)

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, roc_auc_score,
                             confusion_matrix, ConfusionMatrixDisplay,
                             precision_recall_fscore_support)
import xgboost as xgb
import re


## 2. Carga dos Dados


In [ ]:
# Fonte (URL RAW no GitHub)
URL = "https://raw.githubusercontent.com/datasciencedonut/Olympic-Swimming-History-1912-to-2020-/main/Olympic_Swimming.csv"

df = pd.read_csv(URL)
print("Dimensões:", df.shape)
df.head()


## 3. EDA mínima


In [ ]:
df.info()
print("\nNulos por coluna:\n", df.isnull().sum())
print("\nDistribuição de Rank (1=ouro, 2=prata, 3=bronze):")
print(df["Rank"].value_counts().sort_index())


## 4. Engenharia de Atributos — Target & Distância


In [ ]:
# Alvo: Medalha = 1 se Rank <=3, senão 0
df['Medalha'] = (df['Rank'] <= 3).astype(int)

# Distância numérica (inclui revezamentos 4x100 -> 400)
def convert_distance(x: str) -> int:
    x = str(x)
    if "x" in x:  # revezamento (ex: 4x100)
        a,b = x.split("x")
        return int(a) * int(b)
    return int(re.sub("[^0-9]", "", x))

df['Distance_value'] = df['Distance (in meters)'].apply(convert_distance)

# Checagens rápidas
print(df[['Distance (in meters)', 'Distance_value']].head(8))
print("\nProporção da classe Medalha:\n", df['Medalha'].value_counts(normalize=True).rename('proportion'))


## 5. Split & Pipeline de Pré-processamento


In [ ]:
features = ['Year', 'Distance (in meters)', 'Stroke', 'Relay?', 'Gender', 'Team', 'Distance_value']
X = df[features].copy()
y = df['Medalha']

cat_cols = ['Stroke', 'Gender', 'Team']
num_cols = ['Year', 'Relay?', 'Distance_value']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=RANDOM_STATE, stratify=y
)

print("Treino/Teste:", X_train.shape, X_test.shape)


## 6. Baseline — Regressão Logística


In [ ]:
logreg = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_prob = logreg.predict_proba(X_test)[:,1]

print("Acurácia:", round(accuracy_score(y_test, y_pred),3))
print(classification_report(y_test, y_pred))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob),3))


## 7. Comparação de Modelos — LogReg × RandomForest × XGBoost


In [ ]:
modelos = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE)
}

resultados = {}
for nome, clf in modelos.items():
    pipe = Pipeline([('preprocessor', preprocessor), ('classifier', clf)])
    pipe.fit(X_train, y_train)
    yp = pipe.predict(X_test)
    ypb = pipe.predict_proba(X_test)[:,1]
    d = classification_report(y_test, yp, output_dict=True)
    resultados[nome] = {
        "Acurácia": accuracy_score(y_test, yp),
        "Precisão (classe 1)": d['1']['precision'],
        "Recall (classe 1)": d['1']['recall'],
        "F1 (classe 1)": d['1']['f1-score'],
        "ROC-AUC": roc_auc_score(y_test, ypb)
    }

pd.DataFrame(resultados).T.sort_values("ROC-AUC", ascending=False)


## 8. Tuning — XGBoost (RandomizedSearchCV)


In [ ]:
param_dist = {
    "classifier__n_estimators": [200, 300, 500, 800],
    "classifier__max_depth": [3, 4, 5, 6, 8],
    "classifier__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "classifier__subsample": [0.7, 0.8, 0.9, 1.0],
    "classifier__colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "classifier__gamma": [0, 1, 5],
    "classifier__min_child_weight": [1, 3, 5],
    "classifier__reg_lambda": [1, 5, 10]
}

xgb_base = xgb.XGBClassifier(eval_metric='logloss', random_state=RANDOM_STATE, tree_method='hist')

pipe_xgb = Pipeline([('preprocessor', preprocessor), ('classifier', xgb_base)])

search = RandomizedSearchCV(
    pipe_xgb, param_distributions=param_dist, n_iter=30, scoring="f1",
    cv=5, random_state=RANDOM_STATE, n_jobs=-1, verbose=1
)
search.fit(X_train, y_train)
best_model = search.best_estimator_
print("Melhor combinação:", search.best_params_)

y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

print("Acurácia:", round(accuracy_score(y_test, y_pred),3))
print(classification_report(y_test, y_pred))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob),3))

cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_).plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão - XGBoost (tuned, thr=0.50)")
plt.show()


## 9. Ajuste de Threshold (decisão)


In [ ]:
def avalia_threshold(y_true, y_scores, thresholds):
    rows = []
    for t in thresholds:
        yp = (y_scores >= t).astype(int)
        prec, rec, f1, _ = precision_recall_fscore_support(y_true, yp, average='binary')
        rows.append([t, prec, rec, f1])
    return pd.DataFrame(rows, columns=["threshold","precision_1","recall_1","f1_1"])

ths = np.linspace(0.20, 0.80, 25)
thr_df = avalia_threshold(y_test, y_prob, ths)
thr_df

# Aplicar threshold final
THRESHOLD = 0.40
y_pred_thr = (y_prob >= THRESHOLD).astype(int)

print(f"\nThreshold fixado em {THRESHOLD}")
print("Acurácia:", round(accuracy_score(y_test, y_pred_thr),3))
print(classification_report(y_test, y_pred_thr))

cm = confusion_matrix(y_test, y_pred_thr)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_).plot(cmap=plt.cm.Blues)
plt.title(f"Matriz - XGBoost (threshold={THRESHOLD})")
plt.show()


## 10. Importância das Variáveis (XGBoost)


In [ ]:
# Extrair nomes de features após o preprocessor
feat_names = best_model.named_steps['preprocessor'].get_feature_names_out()
importances = best_model.named_steps['classifier'].feature_importances_

fi = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False)
display(fi.head(20))

# Plot simples com matplotlib (top 15)
topk = fi.head(15)[::-1]  # invert for horizontal plot
plt.figure(figsize=(8,6))
plt.barh(topk["feature"], topk["importance"])
plt.title("Importância das variáveis — XGBoost")
plt.tight_layout()
plt.show()


## 11. Experimentos de Robustez (opcional)


In [ ]:
# A) Sem Team
Xb = df[['Year','Distance (in meters)','Stroke','Relay?','Gender','Distance_value']].copy()
yb = df['Medalha']

cat_b = ['Stroke','Gender']
num_b = ['Year','Relay?','Distance_value']

pre_b = ColumnTransformer([('num', StandardScaler(), num_b),
                           ('cat', OneHotEncoder(handle_unknown='ignore'), cat_b)])

Xtr_b, Xte_b, ytr_b, yte_b = train_test_split(Xb, yb, test_size=0.30, random_state=RANDOM_STATE, stratify=yb)
pipe_b = Pipeline([('pre', pre_b), ('clf', xgb.XGBClassifier(eval_metric='logloss', random_state=RANDOM_STATE, tree_method='hist'))])
pipe_b.fit(Xtr_b, ytr_b)
yp_b  = pipe_b.predict(Xte_b)
ypb_b = pipe_b.predict_proba(Xte_b)[:,1]

print("Acurácia (sem Team):", round(accuracy_score(yte_b, yp_b),3))
print(classification_report(yte_b, yp_b))
print("ROC-AUC:", round(roc_auc_score(yte_b, ypb_b),3))

# B) Agrupar Team: Top-N vs OTHER (definido no TREINO)
Xg = df[['Year','Distance (in meters)','Stroke','Relay?','Gender','Team','Distance_value']].copy()
yg = df['Medalha']
Xtr_g, Xte_g, ytr_g, yte_g = train_test_split(Xg, yg, test_size=0.30, random_state=RANDOM_STATE, stratify=yg)

medals_by_team = (pd.DataFrame({"Team": Xtr_g['Team'].values, "Medalha": ytr_g.values})
                  .groupby('Team')['Medalha'].mean().sort_values(ascending=False))
top_teams = set(medals_by_team.head(TOP_N).index)

def map_team(team): return team if team in top_teams else "OTHER"
Xtr_g['Team_grouped'] = Xtr_g['Team'].map(map_team)
Xte_g['Team_grouped']  = Xte_g['Team'].map(map_team)

cat_g = ['Stroke','Gender','Team_grouped']
num_g = ['Year','Relay?','Distance_value']
pre_g = ColumnTransformer([('num', StandardScaler(), num_g),
                           ('cat', OneHotEncoder(handle_unknown='ignore'), cat_g)])

pipe_g = Pipeline([('pre', pre_g), ('clf', xgb.XGBClassifier(eval_metric='logloss', random_state=RANDOM_STATE, tree_method='hist',
                                                             n_estimators=300, max_depth=6, learning_rate=0.1,
                                                             subsample=0.7, colsample_bytree=1.0, gamma=1, min_child_weight=3, reg_lambda=10))])
pipe_g.fit(Xtr_g, ytr_g)
yp_g  = pipe_g.predict(Xte_g)
ypb_g = pipe_g.predict_proba(Xte_g)[:,1]

print("\nAcurácia (Top-N países):", round(accuracy_score(yte_g, yp_g),3))
print(classification_report(yte_g, yp_g))
print("ROC-AUC:", round(roc_auc_score(yte_g, ypb_g),3))

# Threshold aplicado (0.40)
yp_thr_g = (ypb_g >= THRESHOLD).astype(int)
print("\nCom threshold=0.40:")
print("Acurácia:", round(accuracy_score(yte_g, yp_thr_g),3))
print(classification_report(yte_g, yp_thr_g))


## 12. Resultados & Conclusões

- **Modelo final:** **XGBoost** (com *tuning*) + **threshold = 0.40** (prioriza recall de medalhistas).
- **Principais métricas esperadas (referência):** acurácia ~0.62, ROC-AUC ~0.65, recall(classe 1) ~0.55.
- **Variáveis mais relevantes:** `Team_*` (país, especialmente EUA), `Relay?`, `Stroke`, `Distance_value`, `Year`.
- **Trade-off controlado pelo threshold:** aumentar recall de medalhistas implica reduzir precisão/ acurácia — decisão intencional do negócio.

**Limitações e melhorias:**
1) Dependência de `Team` (tradição histórica) — avaliar *target encoding* com CV.  
2) Validação temporal (treinar em anos antigos, testar em anos recentes).  
3) Calibração de probabilidades (Platt/Isotônica) e análise por evento específico.


## 13. Checklist (atendido)

- **Definição do problema:** clara e com hipóteses.  
- **Preparação de dados:** split estratificado; engenharia (`Distance_value`); one-hot + padronização.  
- **Modelagem:** baseline + comparação (LogReg, RF, XGB); *tuning* por CV.  
- **Otimização:** RandomizedSearchCV com espaço e melhores parâmetros registrados.  
- **Avaliação:** métricas adequadas; matriz de confusão; *threshold sweep* documentado.  
- **Boas práticas:** seeds; pipeline; execução fim-a-fim via URL pública; documentação por seção.
